In [23]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [24]:
feedback = pd.read_csv(
    r"D:\vs_code\GUVI\blinkit_project\data_raw\Blinkit - blinkit_customer_feedback.csv"
)

# 1 = Positive sentiment, 0 = Neutral / Negative
feedback["is_positive"] = (feedback["sentiment"] == "Positive").astype(int)

data = feedback[["feedback_text", "is_positive"]].dropna()
X = data["feedback_text"]
y = data["is_positive"]

print(y.value_counts())  # just to see class balance


is_positive
0    3380
1    1620
Name: count, dtype: int64


In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [26]:
model = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
        stop_words="english"
    )),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [27]:
y_pred = model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       676
           1       1.00      0.88      0.94       324

    accuracy                           0.96      1000
   macro avg       0.97      0.94      0.95      1000
weighted avg       0.96      0.96      0.96      1000

Confusion Matrix:
[[676   0]
 [ 39 285]]


In [28]:
test_sentences = [
    "delivery was quick",
    "very happy with the service",
    "excellent quality and super fast delivery",
    "worst experience ever",
    "items were missing from my order",
    "delivery was late and I am disappointed"
]

for s in test_sentences:
    print(s, "->", model.predict([s])[0])


delivery was quick -> 0
very happy with the service -> 1
excellent quality and super fast delivery -> 1
worst experience ever -> 0
items were missing from my order -> 0
delivery was late and I am disappointed -> 0


In [29]:
import joblib, os

model_dir = r"D:\vs_code\GUVI\blinkit_project\data_processed"
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, "feedback_sentiment_model.joblib")
joblib.dump(model, model_path)

print("Saved model to:", model_path)


Saved model to: D:\vs_code\GUVI\blinkit_project\data_processed\feedback_sentiment_model.joblib


In [30]:
model_path = r"D:\vs_code\GUVI\blinkit_project\data_processed\feedback_sentiment_model_v2.joblib"
joblib.dump(model, model_path)


['D:\\vs_code\\GUVI\\blinkit_project\\data_processed\\feedback_sentiment_model_v2.joblib']